# Tensorflow MNIST Model Deployment

 * Wrap a Tensorflow MNIST python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Depenencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install tensorflow
pip install grpcio-tools
```

## Train locally
 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets, metrics
from sklearn.utils import shuffle
from sklearn.datasets import fetch_mldata
from sklearn.externals import joblib

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

mnist_images = mnist.train.images
mnist_labels = mnist.train.labels
 # To apply a classifier on this data, we need to flatten the image, to
    # turn the data in a (samples, feature) matrix:
n_samples = len(mnist_images)
data = mnist_images.reshape((n_samples, -1))
targets = mnist_labels

data,targets = shuffle(data,targets)
classifier = RandomForestClassifier(n_estimators=30)

# We learn the digits on the first half of the digits
classifier.fit(data[:n_samples // 2], targets[:n_samples // 2])

# Now predict the value of the digit on the second half:
expected = targets[n_samples // 2:]
test_data = data[n_samples // 2:]

print(classifier.score(test_data, expected))

predicted = classifier.predict(data[n_samples // 2:])

print("Classification report for classifier %s:\n%s\n"
          % (classifier, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

joblib.dump(classifier, 'sk.pkl')



Wrap model using s2i

In [ ]:
!s2i build . seldonio/seldon-core-s2i-python3:0.2 sk-mnist:0.1

In [ ]:
!docker run --name "mnist_predictor" -d --rm -p 5000:5000 sk-mnist:0.1

In [ ]:
!cd ../../../wrappers/testing && make build_protos

Send some random features that conform to the contract

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p

In [ ]:
!docker rm mnist_predictor --force

In [ ]:
!minikube start --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

In [ ]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3:0.2 sk-mnist:0.1

In [ ]:
!kubectl create -f sk_mnist.json

Wait until ready (replicas == replicasAvailable)

In [ ]:
!kubectl get seldondeployments sk-mnist -o jsonpath='{.status}' 

In [ ]:
!cd ../../../util/api_tester && make build_protos 

In [ ]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

In [ ]:
!minikube delete